In [1]:
from dotenv import load_dotenv
load_dotenv()

True

#  Calling

## __ call __

In [8]:
from langchain import PromptTemplate, LLMChain, OpenAI
from langchain.chat_models import ChatOpenAI

prompt_template = "What is the word to replace the following: {word}"

llm = OpenAI(model_name='gpt-3.5-turbo', temperature=0)

llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))

llm_chain("artificial")

/Users/charanmannuru/miniconda3/envs/LLM/lib/python3.10/site-packages/langchain/llms/openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/charanmannuru/miniconda3/envs/LLM/lib/python3.10/site-packages/langchain/llms/openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


{'word': 'artificial', 'text': 'synthetic'}

## Apply

In [10]:
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

# llm_chain.apply(input_list) # batch processing only for some models.

## Generate

In [ ]:
llm_chain.generate(input_list)

## Predict

### Multiple

In [13]:
prompt_template = "Looking at the context of '{context}'. What is a approapriate word to replace the following; {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=['word', 'context'])
)

llm_chain.predict(word='fan', context='object')

'air conditioner'

In [14]:
llm_chain.predict(word="fan", context="humans")

'supporter'

### from String

In [15]:
template = """Looking at the context of '{context}'. What is a approapriate word to replace the following: {word}?"""
llm_chain = LLMChain.from_string(llm=llm, template=template)

In [16]:
llm_chain.predict(word="fan", context="object")

'air conditioner'

## Parsers

In [17]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template= """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=[], output_parser=output_parser)
)

llm_chain.predict()

'synthetic, man-made, fake, simulated, imitation, faux, counterfeit, ersatz, fabricated, manufactured'

In [18]:
llm_chain.predict_and_parse()

/Users/charanmannuru/miniconda3/envs/LLM/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


['synthetic',
 'man-made',
 'fake',
 'simulated',
 'imitation',
 'faux',
 'ersatz',
 'fabricated',
 'counterfeit',
 'pseudo']

# Adding Memory Conversational Chain

In [21]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory()
)

conversation.predict_and_parse(input="Answer briefly. write the first 3 options.")


/Users/charanmannuru/miniconda3/envs/LLM/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


['synthetic', 'man-made', 'simulated']

In [22]:
conversation.predict_and_parse(input="And the next 4?")

/Users/charanmannuru/miniconda3/envs/LLM/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


['AI: fake', 'imitation', 'faux', 'counterfeit']